# AWS S3 Data Management Tampering

Detects when a user tampers with S3 data management in Amazon Web Services.

References:
* https://github.com/elastic/detection-rules/pull/1145/files
* https://docs.aws.amazon.com/AmazonS3/latest/API/API_Operations.html
* https://docs.aws.amazon.com/AmazonS3/latest/API/API_PutBucketLogging.html
* https://docs.aws.amazon.com/AmazonS3/latest/API/API_PutBucketWebsite.html
* https://docs.aws.amazon.com/AmazonS3/latest/API/API_PutBucketEncryption.html
* https://docs.aws.amazon.com/AmazonS3/latest/userguide/setting-repl-config-perm-overview.html
* https://docs.aws.amazon.com/AmazonS3/latest/API/API_RestoreObject.html

False Positives:
* A S3 configuration change may be done by a system or network administrator. Verify whether the user identity, user agent, and/or hostname should be making changes in your environment. S3 configuration change from unfamiliar users or hosts should be investigated. If known behavior is causing false positives, it can be exempted from the rule.

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
eventSource="s3.amazonaws.com"
eventName=(
PutBucketLogging
PutBucketWebsite
PutEncryptionConfiguration
PutLifecycleConfiguration
PutReplicationConfiguration
ReplicateObject
RestoreObject
)
| stats
min(@scnr.datetime) as firstTime,
max(@scnr.datetime) as lastTime,
count() as eventCount
by
userIdentity.arn,
eventSource,
eventName,
awsRegion

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName